# Exercise 4.9-1

## Task 3 Import libraries and new customer data

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Set path
path = r"C:\Users\tiffk\Instacart Basket Analysis 22-05-2024"

In [4]:
# Import new data
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
# Check customer data
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_customers.shape

(206209, 10)

## Task 4 Wrangle data to follow consistent logic; rename & drop columns as needed

In [6]:
# Customer first & last names not needed for analysis
df_customers.drop(columns = ['First Name', 'Surnam'], axis=1, inplace = True)

In [7]:
# Check that columns were dropped
df_customers.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# normalize column naming
df_customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age'}, inplace = True)

In [9]:
# Creating df of all rows where age column is NaN
df_nan = df_customers[df_customers["age"].isnull() == True]

In [10]:
df_nan

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income


In [11]:
# Creating new products df free of NaN values
df_customers_clean = df_customers[df_customers["age"].isnull() == False]

In [12]:
# Double-check column names
df_customers.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


## Task 5 Fundamental data quality & consistency checks; check for missing values & duplicates, convert mixed-type data

In [13]:
# Convert mixed data types
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis=1)
    if len(df_customers[weird]) > 0:
        print(col + ' is mixed')
    else:
        print(col + ' is consistent')

user_id is consistent
gender is consistent
state is consistent
age is consistent
date_joined is consistent
n_dependants is consistent
fam_status is consistent
income is consistent


In [14]:
# Check for missing values
df_customers.isnull().sum()

user_id         0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [15]:
# Check duplicate columns
df_dups = df_customers[df_customers.duplicated()]

In [16]:
df_dups

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income


No duplicate rows, mixed data-type columns, or missing values found

In [18]:
# Max & Min values
df_customers.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


## Task 6 Combine customer data with prepared Instacart data

In [19]:
# Check datatypes
df_customers.dtypes

user_id          int64
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [5]:
# Change some int64 datatypes to smaller types to aid memory
df_customers = df_customers.astype({'user_id' : 'int32', 'age' : 'int8', 'n_dependants' : 'int8', 'income' : 'int32'})

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'age' not found in columns."

This will also be helpful since I previously changed the datatype of the "user_id" column in the ords_prods_merge data to int 32 and these will be the key columns. int32 was chosen because i discovered it saved memory space better than making it a str.

In [21]:
# Check changes
df_customers.dtypes

user_id          int32
gender          object
state           object
age               int8
date_joined     object
n_dependants      int8
fam_status      object
income           int32
dtype: object

In [23]:
# Import ords_prods_merge data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_v4.pkl'))

In [24]:
# Check imported data
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,user_mean_product_price,spending_flag,median_days_between_orders,frequency_flag
92368,196,Soda,77,7,9.0,2539329,1,1,2,8,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7157228,12427,Original Beef Jerky,23,19,4.4,2539329,1,1,2,8,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8435031,14084,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,2539329,1,1,2,8,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
16637655,26088,Aged White Cheddar Popcorn,23,19,4.7,2539329,1,1,2,8,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
17004958,26405,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,2539329,1,1,2,8,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [25]:
# Change name of merge flag to allow for new merge indicator
ords_prods_merge.rename(columns = {'_merge' : 'merge_1'}, inplace = True)

In [29]:
# Check for change
ords_prods_merge.dtypes

product_id                       int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
order_id                         int64
user_id                          int64
order_number                     int64
orders_day_of_week               int64
order_hour_of_day                int64
days_since_prior_order         float64
first_order                       bool
add_to_cart_order                int64
reordered                        int64
merge_1                       category
price_range_loc                 object
busiest_days                    object
busiest_period_of_day           object
max_order                        int64
loyalty_flag                    object
user_mean_product_price        float64
spending_flag                   object
median_days_between_orders     float64
frequency_flag                  object
dtype: object

In [30]:
ords_prods_merge = ords_prods_merge.astype({'user_id' : 'int32', 'order_id' : 'int32', 'order_number' : 'int32', 'max_order' : 'int32'})

In [31]:
# Check for change
ords_prods_merge.dtypes

product_id                       int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
order_id                         int32
user_id                          int32
order_number                     int32
orders_day_of_week               int64
order_hour_of_day                int64
days_since_prior_order         float64
first_order                       bool
add_to_cart_order                int64
reordered                        int64
merge_1                       category
price_range_loc                 object
busiest_days                    object
busiest_period_of_day           object
max_order                        int32
loyalty_flag                    object
user_mean_product_price        float64
spending_flag                   object
median_days_between_orders     float64
frequency_flag                  object
dtype: object

In [32]:
ords_prods_merge = ords_prods_merge.astype({'product_id' : 'int32', 'aisle_id' : 'int32', 'department_id' : 'int32', 'order_hour_of_day' : 'int32'})

In [33]:
# Check for change
ords_prods_merge.dtypes

product_id                       int32
product_name                    object
aisle_id                         int32
department_id                    int32
prices                         float64
order_id                         int32
user_id                          int32
order_number                     int32
orders_day_of_week               int64
order_hour_of_day                int32
days_since_prior_order         float64
first_order                       bool
add_to_cart_order                int64
reordered                        int64
merge_1                       category
price_range_loc                 object
busiest_days                    object
busiest_period_of_day           object
max_order                        int32
loyalty_flag                    object
user_mean_product_price        float64
spending_flag                   object
median_days_between_orders     float64
frequency_flag                  object
dtype: object

In [35]:
# Merge dataframes on user_id column. Check to see how this will look first:
pd.merge(df_customers, ords_prods_merge, on = ['user_id'], indicator = True)

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income,product_id,product_name,...,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,user_mean_product_price,spending_flag,median_days_between_orders,frequency_flag,_merge
0,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Low-range product,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
1,26711,Female,Missouri,48,1/1/2017,3,married,165665,38928,0% Greek Strained Yogurt,...,Mid-range product,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
2,26711,Female,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,...,Low-range product,Slowest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
3,26711,Female,Missouri,48,1/1/2017,3,married,165665,38928,0% Greek Strained Yogurt,...,Mid-range product,Slowest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
4,26711,Female,Missouri,48,1/1/2017,3,married,165665,47402,Fuji Apples,...,Mid-range product,Slowest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32399727,80148,Female,New York,55,4/1/2020,1,married,57095,39972,Chocolate Fudge High Protein Bar,...,Low-range product,Regularly busy,Most orders,4,New customer,3.886667,Low spender,12.0,Regular customer,both
32399728,80148,Female,New York,55,4/1/2020,1,married,57095,41427,Cookies 'N Cream Protein Bar,...,Low-range product,Regularly busy,Most orders,4,New customer,3.886667,Low spender,12.0,Regular customer,both
32399729,80148,Female,New York,55,4/1/2020,1,married,57095,48335,Chocolate Brownie Energy Bar,...,Mid-range product,Regularly busy,Most orders,4,New customer,3.886667,Low spender,12.0,Regular customer,both
32399730,80148,Female,New York,55,4/1/2020,1,married,57095,3198,Pumpkin Spice Latte Iced Coffee Pure Cold Brew...,...,Low-range product,Regularly busy,Average orders,4,New customer,3.886667,Low spender,12.0,Regular customer,both


In [36]:
# Go ahead with merging dataframes
ords_prods_all = ords_prods_merge.merge(df_customers, on = ['user_id'], indicator = True)

In [37]:
# Rename and check new merge flag
ords_prods_all.rename(columns = {'_merge' : 'merge_2'}, inplace = True)

In [38]:
ords_prods_all['merge_2'].value_counts(dropna = False)

merge_2
both          32399732
left_only            0
right_only           0
Name: count, dtype: int64

In [40]:
# Check new dataframe as a whole
ords_prods_all.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,median_days_between_orders,frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income,merge_2
0,196,Soda,77,7,9.0,2539329,1,1,2,8,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,12427,Original Beef Jerky,23,19,4.4,2539329,1,1,2,8,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,14084,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,2539329,1,1,2,8,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,26088,Aged White Cheddar Popcorn,23,19,4.7,2539329,1,1,2,8,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,26405,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,2539329,1,1,2,8,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [41]:
# allow for viewing of middle columns
pd.options.display.max_columns = None

In [42]:
ords_prods_all.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,merge_1,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,user_mean_product_price,spending_flag,median_days_between_orders,frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income,merge_2
0,196,Soda,77,7,9.0,2539329,1,1,2,8,NaN,True,1,0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,12427,Original Beef Jerky,23,19,4.4,2539329,1,1,2,8,NaN,True,3,0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,14084,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,2539329,1,1,2,8,NaN,True,2,0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,26088,Aged White Cheddar Popcorn,23,19,4.7,2539329,1,1,2,8,NaN,True,4,0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,26405,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,2539329,1,1,2,8,NaN,True,5,0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


## Task 8 Export new dataframe as pickle file for part 2

In [43]:
# Exporting data
ords_prods_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))